# Importing libraries

In [2]:
import pandas as pd
import numpy as np
import math
import os
import sys
import json
import networkx as nx

# Current and parent directories

In [4]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)

# Read zone information

In [6]:
zones = pd.read_excel(str(parent_directory) + '/data/Aras-Information.xlsx', sheet_name='Zone-Info')
num_zones = len(zones)

# Parameters

In [8]:
num_timeslots = 1440
num_zones = 5

# Save linearized attack schedule (STRENGTH) for all houses and occupants

In [10]:
strength_attack_costs_actual = dict()

adm_algo = 'DBSCAN'
for house_name in ['A']:
    for occupant_id in ['1', '2']:
        print("Linearized: House: " + str(house_name) + ", Occupant: " + str(occupant_id) + ", ADM Algo: " + str(adm_algo))                  

        filename = str(parent_directory) + '/data/deadlock-elimination/Linearized_' + str(adm_algo) + '_House-' + str(house_name) + '_Occupant-' + str(occupant_id) + '.json'

        with open(filename, "r") as file:
            data = json.load(file)

        rewards = [0, 1, 2, 4, 3]
        states = data["States"]
        next_states = data["Next-States"]

        my_graph = nx.DiGraph()         
        for i in range(len(states)):
            state = states[i]
            arrival_time = int(state.split('-')[0])
            arrival_zone = int(state.split('-')[1])
            state_stay_duration = int(state.split('-')[2])
            try:
                for j in next_states[str(i)]:
                    if j == -1:
                        my_graph.add_edge(state, '0-0-0', weight = 0)
                        continue

                    next_state = states[j]
                    next_arrival_time = int(next_state.split('-')[0])
                    next_arrival_zone = int(next_state.split('-')[1])
                    next_state_stay_duration = int(next_state.split('-')[2])

                    if next_arrival_zone == arrival_zone:
                        my_graph.add_edge(state, next_state, weight = (next_state_stay_duration - state_stay_duration) * rewards[arrival_zone])
                    else:
                        my_graph.add_edge(state, next_state, weight = next_state_stay_duration  * rewards[next_arrival_zone])
            except:
                pass

        source_node = '0-0-0'

        current_source_nodes = [node for node in my_graph.nodes if my_graph.in_degree(node) == 0]

        # remove nodes whose arrival_time! = 0 and does not have any preceeding node
        nodes_to_remove = []

        for node in current_source_nodes:
            arrival_time = int(node.split('-')[0])
            if arrival_time != 0:
                nodes_to_remove.append(node)

        current_source_nodes = [node for node in current_source_nodes if node not in nodes_to_remove]

        for node in current_source_nodes:
            node_state_stay_duration = int(node.split('-')[2])
            my_graph.add_edge(source_node, node, weight = node_state_stay_duration)

        # Get all nodes reachable from the source node
        reachable_nodes = nx.descendants(my_graph, source_node)
        reachable_nodes.add(source_node)

        # Create a subgraph containing only the reachable nodes
        my_graph = my_graph.subgraph(reachable_nodes)    

        longest_path = nx.dag_longest_path(my_graph, weight = 'weight')    
        len_longest_path = nx.dag_longest_path_length(my_graph, weight = 'weight')

        attack_schedule = {"Longest-Path" : longest_path, "Longest-Path-Length" : len_longest_path, "Number-of-Nodes" : my_graph.number_of_nodes(), "Number-of-Edges:" : my_graph.number_of_edges()}

        output_filename = str(parent_directory) + '/data/attack-schedules/strength/Linearized_' + str(adm_algo) + '_House-' + str(house_name) + '_Occupant-' + str(occupant_id) + '.json'

        with open(output_filename, "w") as json_file:
            json.dump(attack_schedule, json_file)

Linearized: House: A, Occupant: 1, ADM Algo: DBSCAN
Linearized: House: A, Occupant: 2, ADM Algo: DBSCAN


# Save actual attack schedule (STRENGTH) for all houses and occupants

In [12]:
strength_attack_costs_actual = dict()

adm_algo = 'DBSCAN'
for house_name in ['A']:
    for occupant_id in ['1', '2']:
        print("Actual: House: " + str(house_name) + ", Occupant: " + str(occupant_id) + ", ADM Algo: " + str(adm_algo))                  

        filename = str(parent_directory) + '/data/deadlock-elimination/Actual_' + str(adm_algo) + '_House-' + str(house_name) + '_Occupant-' + str(occupant_id) + '.json'

        with open(filename, "r") as file:
            data = json.load(file)

        rewards = [0, 1, 2, 4, 3]
        states = data["States"]
        next_states = data["Next-States"]

        my_graph = nx.DiGraph()         
        for i in range(len(states)):
            state = states[i]
            arrival_time = int(state.split('-')[0])
            arrival_zone = int(state.split('-')[1])
            state_stay_duration = int(state.split('-')[2])
            try:
                for j in next_states[str(i)]:
                    if j == -1:
                        my_graph.add_edge(state, '0-0-0', weight = 0)
                        continue

                    next_state = states[j]
                    next_arrival_time = int(next_state.split('-')[0])
                    next_arrival_zone = int(next_state.split('-')[1])
                    next_state_stay_duration = int(next_state.split('-')[2])

                    if next_arrival_zone == arrival_zone:
                        my_graph.add_edge(state, next_state, weight = (next_state_stay_duration - state_stay_duration) * rewards[arrival_zone])
                    else:
                        my_graph.add_edge(state, next_state, weight = next_state_stay_duration  * rewards[next_arrival_zone])
            except:
                pass

        source_node = '0-0-0'

        current_source_nodes = [node for node in my_graph.nodes if my_graph.in_degree(node) == 0]

        # remove nodes whose arrival_time! = 0 and does not have any preceeding node
        nodes_to_remove = []

        for node in current_source_nodes:
            arrival_time = int(node.split('-')[0])
            if arrival_time != 0:
                nodes_to_remove.append(node)

        current_source_nodes = [node for node in current_source_nodes if node not in nodes_to_remove]

        for node in current_source_nodes:
            node_state_stay_duration = int(node.split('-')[2])
            my_graph.add_edge(source_node, node, weight = node_state_stay_duration)

        # Get all nodes reachable from the source node
        reachable_nodes = nx.descendants(my_graph, source_node)
        reachable_nodes.add(source_node)

        # Create a subgraph containing only the reachable nodes
        my_graph = my_graph.subgraph(reachable_nodes)    

        longest_path = nx.dag_longest_path(my_graph, weight = 'weight')    
        len_longest_path = nx.dag_longest_path_length(my_graph, weight = 'weight')

        attack_schedule = {"Longest-Path" : longest_path, "Longest-Path-Length" : len_longest_path, "Number-of-Nodes" : my_graph.number_of_nodes(), "Number-of-Edges:" : my_graph.number_of_edges()}

        output_filename = str(parent_directory) + '/data/attack-schedules/strength/Actual_' + str(adm_algo) + '_House-' + str(house_name) + '_Occupant-' + str(occupant_id) + '.json'

        with open(output_filename, "w") as json_file:
            json.dump(attack_schedule, json_file)

Actual: House: A, Occupant: 1, ADM Algo: DBSCAN
Actual: House: A, Occupant: 2, ADM Algo: DBSCAN
